# Generate all figures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hypothesis as h
from itertools import cycle

import time

In [ ]:
from plotting_tools import make_dirs_safe
directory = 'figures/'
make_dirs_safe(directory)
lines = ["-", "--", "-.", ":"]
linecycler = cycle(lines)

In [ ]:
from trajectory_creator import get_trajectory

np.random.seed(0)

fig, axis = plt.subplots(1, 3, figsize=(6, 2), sharey='row',
                        gridspec_kw={'hspace': 0, 'wspace': 0.1})
traj = get_trajectory("clover")
traj.scale_bounding_box([5, 5])
error = np.random.normal(size=traj.coeffs.shape)
error = error/np.linalg.norm(error)
# todo scale the trajectory
for idx, scale in enumerate([0.1, 0.5, 1]):
    ax = axis[idx]
    traj.plot(ax=ax, linestyle='--')
    perturbed = traj.copy()
    perturbed.coeffs += scale*error
    perturbed.plot(ax=ax)
    ax.axis('equal')
    ax.set_title("error {}m".format(scale))
    if idx==0:
        ax.set_ylabel("distance [m]")
plt.tight_layout()
plt.savefig(directory+"error_overview.pdf")

In [ ]:
from plotting_tools import plot_noise
plt.rcParams['figure.figsize'] = 5, 2.5

In [ ]:
plot = plot_noise('noise_and_anchors', error_types=['errors'], save_figures=True,  min_noise=0, max_noise=None, background_alpha=0.3, start=30, anchors=True)
plt.ylim(5e-6)
plt.savefig(directory+'noise_and_anchors.pdf')
plt.show()

In [ ]:
plot = plot_noise('noise_to_square_right_inverse', error_types=['errors'], save_figures=False,  max_noise=None, background_alpha=0.2, start=100)
plt.savefig(directory+'noise_to_quare_error.pdf')
plt.ylim(1e-4, 1e2)
plt.show()

In [ ]:
plot = plot_noise('noise_right_inverse', error_types=['errors'], save_figures=True,  max_noise=None, background_alpha=0.3, start=30)
plt.savefig(directory+'noise_error.pdf')
plt.ylim(1e-4, 1e2)
plt.show()

In [ ]:
plot_noise('noise_right_inverse_weighted', error_types=['errors'], save_figures=True,  max_noise=None, background_alpha=0.3, start=30)
plt.savefig(directory+'noise_weighted_error.pdf')
plt.ylim(1e-4, 1e2)
plt.show()

In [ ]:
n_measurement_list = range(10, 40)

experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "n_anchors_list": [3, 4, 5],
}

start = time.time()
probabilities = []
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):
    print("{} anchors".format(n_anchors))
    probabilities.append([h.probability_upper_bound_any_measurements(
        n_dimensions=experiment_params["n_dimensions"],
        n_constraints=experiment_params["n_constraints"],
        n_measurements=n,
        position_wise=False,
        n_anchors=n_anchors,
        n_positions=np.Infinity,
        full_matrix=True
    ) for n in n_measurement_list])
probabilities = np.array(probabilities)
print("total time: {:.2f}s".format(time.time()-start))

In [ ]:
linecycler = cycle(lines)
f, ax = plt.subplots()
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):

    p = plt.step(
        n_measurement_list,
        probabilities[idx],
        label="{} anchors".format(n_anchors),
        where='post',
        linestyle=next(linecycler))
plt.xlabel("number of measurements")
plt.ylabel("probability")
plt.ylim(0)
plt.grid()
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig(directory + 'probability_anchors.pdf')

plt.show()

In [ ]:
experiment_params={
    "n_dimensions": 2,
    "n_constraints_list": [3, 5, 7],
    "n_anchors": 4,
}

start = time.time()
probabilities = []
for idx, n_constraints in enumerate(experiment_params["n_constraints_list"]):
    print("{} constraints".format(n_constraints))
    probabilities.append([h.probability_upper_bound_any_measurements(
        n_dimensions=experiment_params["n_dimensions"],
        n_constraints=n_constraints,
        n_measurements=n,
        position_wise=False,
        n_anchors=experiment_params["n_anchors"],
        n_positions=np.Infinity,
        full_matrix=True,
    ) for n in n_measurement_list])
probabilities = np.array(probabilities)
print("total time: {:.2f}s".format(time.time()-start))

In [ ]:
linecycler = cycle(lines)
f, ax = plt.subplots()
for idx, n_constraints in enumerate(experiment_params["n_constraints_list"]):

    p = plt.step(
        n_measurement_list,
        probabilities[idx],
        label="{} d.o.f.".format(n_constraints),
        where='post',
        linestyle=next(linecycler))
plt.xlabel("number of measurements")
plt.ylabel("probability")
plt.ylim(0)
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(directory + 'probability_dof.pdf')

plt.show()